In [42]:
import numpy as np
import pysindy as ps
from pysindy import SINDy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from pysindy.optimizers import STLSQ


Loading the data from the Kaggle dataset

In [22]:
data = np.genfromtxt('/home/luka/Desktop/projekti/sindy_tutorial/archive/reconstruction/train.csv', delimiter=',', skip_header=1)

Extracting the variables from the dataset

In [ ]:
t = data[:, 0]
X = data[:, 1:3]

#time step
dt = t[1] - t[0]

Let's see what SINDy identifies by default. We use the STSLQ as an L1 regularization in the sparse regression.

In [ ]:
model = SINDy(optimizer=STLSQ(threshold=0.1))

# Naming the variables
model.fit(X, t=dt, feature_names=["x", "v"])

model.print()
print("Model score (R^2):", model.score(X, t=dt))

(x)' =  0.658 v + -0.314 x^2 +  0.254 x v
(v)' = -0.758 x + -0.250 x^2 +  0.186 x v
Model score (R^2): 0.9395700172025041


By default, SINDy uses the polinomial library. 

In [ ]:
feature_names = model.get_feature_names()
print("Functions in the Theta library:", feature_names)

Funkcije u Theta biblioteci: ['1', 'x', 'v', 'x^2', 'x v', 'v^2']


Now, let's modify the Theta library (containing basis functions). Alongside polinomials, we will add sin/cos basis (Fourie library). Then, we will see if the identified system is changed:

In [ ]:
from pysindy.feature_library import PolynomialLibrary, FourierLibrary

# Joining the Polynomial and Fourier library
combined_lib = PolynomialLibrary(degree=2) + FourierLibrary(n_frequencies=1)

model = SINDy(feature_library=combined_lib,optimizer=STLSQ(threshold=0.1))
model.fit(X, t=dt, feature_names=["x", "v"])

model.print()
print("Model score (R^2):", model.score(X, t=dt))

(x)' = -2.602 x +  0.428 v + -0.193 x^2 +  0.114 x v +  2.733 sin(1 x) +  0.282 sin(1 v)
(v)' = -0.758 x + -0.250 x^2 +  0.186 x v
Model score (R^2): 0.9677484539973241


We now see, that on the observed scope, SINDy finds the usage of sin(x) justified. It fits more closely, although from physics it seems that it shouldn't be the case (since we now the ODE describing damped string problem).